In [27]:
import os

col_names = []  # stores column names from the first file
data_lines = []  # stores all data

directory = "Comp_data"  # path to make a directory

# To loop through all .txt files in the directory
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)

    # To check if it is a file and ends with .txt
    if os.path.isfile(file_path) and filename.endswith(".txt"):
        with open(file_path, "r") as myfile:
            for idx, line in enumerate(myfile):
                line = line.strip()  
                if not line:  # to skip any empty lines
                    continue

                split_line = line.split("\t") 

                if idx == 0:
                    col_names = split_line
                else:
                    data_lines.append(split_line)
                idx += 1

In [4]:
col_names

['Gene',
 'WildType.Sequence',
 'Mutant.Sequence',
 'mRNA.Expression.WT.Rep1',
 'mRNA.Expression.WT.Rep2',
 'mRNA.Expression.WT.Rep3',
 'mRNA.Expression.Mut.Rep1',
 'mRNA.Expression.Mut.Rep2',
 'mRNA.Expression.Mut.Rep3',
 'Protein.Expression.WT.Rep1',
 'Protein.Expression.WT.Rep2',
 'Protein.Expression.WT.Rep3',
 'Protein.Expression.Mut.Rep1',
 'Protein.Expression.Mut.Rep2',
 'Protein.Expression.Mut.Rep3',
 'CellViability.WT.Rep1',
 'CellViability.WT.Rep2',
 'CellViability.WT.Rep3',
 'CellViability.Mut.Rep1',
 'CellViability.Mut.Rep2',
 'CellViability.Mut.Rep3']

In [7]:
import numpy as np
import pandas as pd

In [10]:
import seaborn as sp 
import matplotlib.pyplot as plt
import matplotlib as mpl

In [18]:
df = pd.DataFrame(data_lines, columns=col_names)

In [20]:
df

,Gene,WildType.Sequence,Mutant.Sequence,mRNA.Expression.WT.Rep1,mRNA.Expression.WT.Rep2,mRNA.Expression.WT.Rep3,mRNA.Expression.Mut.Rep1,mRNA.Expression.Mut.Rep2,mRNA.Expression.Mut.Rep3,Protein.Expression.WT.Rep1,...,Protein.Expression.WT.Rep3,Protein.Expression.Mut.Rep1,Protein.Expression.Mut.Rep2,Protein.Expression.Mut.Rep3,CellViability.WT.Rep1,CellViability.WT.Rep2,CellViability.WT.Rep3,CellViability.Mut.Rep1,CellViability.Mut.Rep2,CellViability.Mut.Rep3
0,Jtkir5r8,TACCATCCCTAGCCAAATGGACATCATGCAGGCCGTTTCTCATTAA...,TACCATCCCTAGCCAAATGGACATCATGCAGGCCGTTTCTCATTAA...,5.455004995294742,3.6523482032823615,7.800439764032981,5.885268373073665,3.806745268257305,6.6744116648019265,22855,...,15247,22869,3574,15264,0.25664713139818596,0.8889434572433982,0.17728863331671263,0.20003424326312247,0.7921275507189697,0.19236223188654147
1,Cairn3a2,GCACTATGGAACAAATCTCCGTAGGATAGCCAGAGTAAATCGGCCT...,GCACTATGGAACAAATCTCCGTAGGATAGCCAGAGTAAATCGGCCT...,25.29017748201274,2.711264847023889,48.099041814601954,25.39605299493825,2.6665130333769658,49.18667987284963,25992,...,31383,26001,27949,31399,0.6335142731392773,0.9568891595345175,0.5184037670251207,0.66432236482412,0.9706430686730598,0.6922761635017513
2,Oyu3a9,GGTACGAGCTCCGTCCTATGTCGACCGGGAGCTGGAGCCCCAGCTG...,TCACAATTCCGGCGTGACCATAATATCCGGGGTAGAACTCAACATC...,5.294363666421411,40565.21880350868,919.9062500850056,0.0014001785597936496,-0.0012798841754592851,0.0003668553065877212,2.6533211586263534,...,2685.4628943088,-0.00021505043980370514,0.0016495313834152326,-0.001009792320070798,0.2391663324581974,0.829309755683558,0.08330828926170375,0,0,0
3,Fab6l6,GGAGGTTCCAAAATAATATCGTCACCCCGGTTAGGTGTGGGTAAAA...,GGAGGTTCCAAAATAATATCGTCACCCCGGTTAGGTGTGGGTAAAA...,2957.536857695853,121.20941897807008,6972.993774352871,2958.819316699249,122.8809836269841,6972.853989762808,47460,...,32980,47464,17232,32986,0.3319958182530629,0.04963065642077069,0.37824903700728496,0.45777626832189966,-0.00235251968205219,0.4150365479978685
4,K7c7,AATCGTTGACGGTGTGTACGCGCCCGACCCGGTGAGATGTTATGGA...,AATCGTTGACGGTGTGTACGCGCCCGACCCGGTGAGATGTTATGGA...,6.795284564789637,4.743195696331062,102.60751791748346,6.10652173555912,4.414489044433421,101.79539614226579,41931,...,21886,41941,19949,21897,0.8149501888824089,0.9355881271627626,0.3902907727205055,0.7368706121882426,0.9942224160953352,0.3232503821480863
5,Kl8,CGACACCGTACTCTGTTGAGTGAACCTCGGCATCTCCCGAAGCTTC...,CGACACCGTACTCTGTTGAGTGAACCTCGGCATCTCCCGAAGCTTC...,19.470389342325124,12.97314224729553,1.4853621109867914,19.31573923628884,13.150434151347012,0.9635672395913896,39038,...,47690,39057,17528,47691,0.8745206362218103,0.47921794586153144,0.3696665553418981,0.9567215190437838,0.4811429475742162,0.3302195059534506
6,Clach6,AAATGACATCCTCCGGGGAACGATGCAGGCCCAGTAAGCCAATACT...,AAATGACATCCTCCGGGGAACGATGCAGGCCCAGTAAGCCAATACT...,19.85790945329143,3.451715063775469,2.631587178785222,17.838609143585714,3.0451754478819675,3.0170496559081337,48988,...,7210,48994,4939,7221,0.9023572425786909,0.5548330505464185,0.9114854785499592,0.8849626419722535,0.6053479394779937,1.0015234211025699
7,Everr1,ATCTAGATGTGAATATATATATTCAAAGTTATATATGAAGTGCCCT...,ATCTAGATGTGAATATATATATTCAAAGTTATATATGAAGTGCCCT...,1.2758525042502942,4.984930388775445,579.9119306443383,1.3733481769258036,4.19560297152881,580.8492121009771,6498,...,23849,6513,8420,23853,0.8756026725855244,0.7281673862154504,0.0698576336703659,0.9768002335694101,0.9151791813739114,0.10825632088901904
8,Ist9a7,AACATCCTTATATATGAATTTTACTGGATTGGTCAAGGAGAGTGCG...,AACATCCTTATATATGAATTTTACTGGATTGGTCAAGGAGAGTGCG...,2.1029739285372715,15.157918245502328,292.5990570278786,2.3780051566765636,15.897262312074648,291.95509568384904,21433,...,8270,21447,20592,8285,0.5618903173837013,0.547005359316078,0.3382083933152621,0.47917119491283483,0.4492673377848404,0.41818217773097854
9,App8,CGGGAAGCTAACTTACTCATAAAGCCCGGTTCGAATAACATTTTGA...,CGGGAAGCTAACTTACTCATAAAGCCCGGTTCGAATAACATTTTGA...,2.0514009205463983,2.426572690927556,2.828440821116234,2.31456679785976,3.3307935609226873